In [ ]:
print("Caesar Cipher")
def caesar(c, n):
    if ord(c)+n > ord("Z"):
        return chr(ord(c)+n - 26)
    else:
        return chr(ord(c)+n)

f = open("C:/Users/su076/OneDrive/Documents/OneDrive/Documents/Assignment/ciphertextA.txt", 'r')
data = f.read()
for i in range(1,26):
    ans =''
    for c in data:
        ans += caesar(c,i)
    if "THE" in ans:
        print("KEY A to " +chr(i+65)+":")
        print(ans.replace(";",""))
f.close()

In [66]:
f = open("C:/Users/su076/OneDrive/Documents/OneDrive/Documents/Assignment/ciphertextB.txt", 'r')
data=f.read()
def friedman(c,k):
    freq=[[0]*26 for i in range(k)]
    for i in range(len(c)):
        freq[i%k][ord(c[i])-65]+=1
    I=L=0
    for i in range(k):
        L+=sum(freq[i])*(sum(freq[i])-1)
        for j in range(26):
            I+=freq[i][j]*(freq[i][j]-1)
    return I/L
length=[0,0]
for i in range(1, 11):
    print("key len "+str(i)+":"+str(friedman(data,i)))
    if length[1]<friedman(data,i):
        length[0]=i
        length[1]=friedman(data,i)
print("final key length:"+str(length[0]))
keylength=int(length[0])

key len 1:0.04393789142666235
key len 2:0.045339675932808084
key len 3:0.04385853012931928
key len 4:0.05585296027075063
key len 5:0.044165046216137024
key len 6:0.045730926779313875
key len 7:0.04621911561320226
key len 8:0.08151569583440253
key len 9:0.04527938342967245
key len 10:0.04557389347336834
final key length:8


In [67]:
Alphabet={1:'A',2:'B',3:'C',4:'D',5:'E',6:'F',7:'G',8:'H',9:'I',10:'J',11:'K',12:'L',13:'M',14:'N',15:'O',16:'P',17:'Q',18:'R',19:'S',20:'T',21:'U',22:'V',23:'W',24:'X',25:'Y',26:'Z'}
def findkey(c, keylength):
    c=list(c)
    b=[]
    for i in range(0,keylength):
        b.append([])
    for i in range(0,len(c)):
        b[i%keylength].append(c[i])
    key=""
    frequency=[]
    for i in range(0,keylength):
        frequency.append([])
    
    for i in range(0, keylength):
        for j in range(1, 26+1):
            a=Alphabet[j]
            frequency[i].append([b[i].count(a),a])
        
        frequency[i].sort(reverse=True)
        si=ord(frequency[i][0][1])-ord('E')
        print((frequency[i][0][1]))
        if si < 0:
            si+=65+11
            key+=chr(si)
            print(key)
        else:
            key+=chr(si+65)

    return key
print(findkey(data, keylength))

W
S
S
R
K
W
B
SOONGSI
P
SOONGSIL


In [14]:
def decrypt(c, key):
    keylength=len(key)
    k_int = [ord(i) for i in key]
    c_int = [ord(i) for i in c]
    ans=''
    for i in range(len(c_int)):
        val = (c_int[i]-k_int[i%keylength])%26
        ans+=chr(val+65)
    return ans

In [1]:
#!/usr/bin/env python3

import collections
import hashlib
import random

EllipticCurve = collections.namedtuple('EllipticCurve', 'name p a b g n h')

curve = EllipticCurve(
    'secp256k1',
    # Field characteristic.
    p=0xfffffffffffffffffffffffffffffffffffffffffffffffffffffffefffffc2f,
    # Curve coefficients.
    a=0,
    b=7,
    # Base point.
    g=(0x79be667ef9dcbbac55a06295ce870b07029bfcdb2dce28d959f2815b16f81798,
       0x483ada7726a3c4655da4fbfc0e1108a8fd17b448a68554199c47d08ffb10d4b8),
    # Subgroup order.
    n=0xfffffffffffffffffffffffffffffffebaaedce6af48a03bbfd25e8cd0364141,
    # Subgroup cofactor.
    h=1,
)


# Modular arithmetic ##########################################################

def inverse_mod(k, p):
    """Returns the inverse of k modulo p.
    This function returns the only integer x such that (x * k) % p == 1.
    k must be non-zero and p must be a prime.
    """
    if k == 0:
        raise ZeroDivisionError('division by zero')

    if k < 0:
        # k ** -1 = p - (-k) ** -1  (mod p)
        return p - inverse_mod(-k, p)

    # Extended Euclidean algorithm.
    s, old_s = 0, 1
    t, old_t = 1, 0
    r, old_r = p, k

    while r != 0:
        quotient = old_r // r
        old_r, r = r, old_r - quotient * r
        old_s, s = s, old_s - quotient * s
        old_t, t = t, old_t - quotient * t

    gcd, x, y = old_r, old_s, old_t

    assert gcd == 1
    assert (k * x) % p == 1

    return x % p


# Functions that work on curve points #########################################

def is_on_curve(point):
    """Returns True if the given point lies on the elliptic curve."""
    if point is None:
        # None represents the point at infinity.
        return True

    x, y = point

    return (y * y - x * x * x - curve.a * x - curve.b) % curve.p == 0


def point_neg(point):
    """Returns -point."""
    assert is_on_curve(point)

    if point is None:
        # -0 = 0
        return None

    x, y = point
    result = (x, -y % curve.p)

    assert is_on_curve(result)

    return result


def point_add(point1, point2):
    """Returns the result of point1 + point2 according to the group law."""
    assert is_on_curve(point1)
    assert is_on_curve(point2)

    if point1 is None:
        # 0 + point2 = point2
        return point2
    if point2 is None:
        # point1 + 0 = point1
        return point1

    x1, y1 = point1
    x2, y2 = point2

    if x1 == x2 and y1 != y2:
        # point1 + (-point1) = 0
        return None

    if x1 == x2:
        # This is the case point1 == point2.
        m = (3 * x1 * x1 + curve.a) * inverse_mod(2 * y1, curve.p)
    else:
        # This is the case point1 != point2.
        m = (y1 - y2) * inverse_mod(x1 - x2, curve.p)

    x3 = m * m - x1 - x2
    y3 = y1 + m * (x3 - x1)
    result = (x3 % curve.p,
              -y3 % curve.p)

    assert is_on_curve(result)

    return result


def scalar_mult(k, point):
    """Returns k * point computed using the double and point_add algorithm."""
    assert is_on_curve(point)

    if k % curve.n == 0 or point is None:
        return None

    if k < 0:
        # k * point = -k * (-point)
        return scalar_mult(-k, point_neg(point))

    result = None
    addend = point

    while k:
        if k & 1:
            # Add.
            result = point_add(result, addend)

        # Double.
        addend = point_add(addend, addend)

        k >>= 1

    assert is_on_curve(result)

    return result


# Keypair generation and ECDSA ################################################

def make_keypair():
    """Generates a random private-public key pair."""
    private_key = random.randrange(1, curve.n)
    public_key = scalar_mult(private_key, curve.g)

    return private_key, public_key


def hash_message(message):
    """Returns the truncated SHA521 hash of the message."""
    message_hash = hashlib.sha512(message).digest()
    e = int.from_bytes(message_hash, 'big')

    # FIPS 180 says that when a hash needs to be truncated, the rightmost bits
    # should be discarded.
    z = e >> (e.bit_length() - curve.n.bit_length())

    assert z.bit_length() <= curve.n.bit_length()

    return z


def sign_message(private_key, message):
    z = hash_message(message)

    r = 0
    s = 0

    while not r or not s:
        k = random.randrange(1, curve.n)
        x, y = scalar_mult(k, curve.g)

        r = x % curve.n
        s = ((z + r * private_key) * inverse_mod(k, curve.n)) % curve.n

    return (r, s)


def verify_signature(public_key, message, signature):
    z = hash_message(message)

    r, s = signature

    w = inverse_mod(s, curve.n)
    u1 = (z * w) % curve.n
    u2 = (r * w) % curve.n

    .x, y = point_add(scalar_mult(u1, curveg),
                     scalar_mult(u2, public_key))

    if (r % curve.n) == (x % curve.n):
        return 'signature matches'
    else:
        return 'invalid signature'


print('Curve:', curve.name)

private, public = make_keypair()
print("Private key:", hex(private))
print("Public key: (0x{:x}, 0x{:x})".format(*public))

msg = b'Hello!'
signature = sign_message(private, msg)

print()
print('Message:', msg)
print('Signature: (0x{:x}, 0x{:x})'.format(*signature))
print('Verification:', verify_signature(public, msg, signature))

msg = b'Hi there!'
print()
print('Message:', msg)
print('Verification:', verify_signature(public, msg, signature))

private, public = make_keypair()

msg = b'Hello!'
print()
print('Message:', msg)
print("Public key: (0x{:x}, 0x{:x})".format(*public))
print('Verification:', verify_signature(public, msg, signature))

Curve: secp256k1
Private key: 0x5757541759f452a1927f26f0cb5beaedc6f850c93a4e56e4c3759999c2569ec0
Public key: (0x37f804f9d467e7b854da0a4d32e717d5fd9602a464d45a9002518a0a29e2bb07, 0x85fdfe637f6784118c9bce5c46e3d26d4aadba66722b90560af99c0c316eeb57)

Message: b'Hello!'
Signature: (0x1c7bf861c2c005c256ba8760b865a74341c4902839d209024bbc20ed7c1cf8d5, 0xd3e4ac357ef30d04726271a493ed91b25bd04c8fe05fb7038d8a98d00af4caf4)
Verification: signature matches

Message: b'Hi there!'
Verification: invalid signature

Message: b'Hello!'
Public key: (0x9806ef8509c7f3367572e2b93d805341d398bbd1d03d5af06fb6b8be370c94b9, 0xa6dd9d8020d64ed002a07d9a505b9a244e3bf0318298d0a7aea4e7f2f6459ff7)
Verification: invalid signature
